In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import requests
import cv2

In [2]:
options = Options()
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
dr = webdriver.Chrome(options=options)

In [3]:
dr.get('https://www.citychain.com.hk/en/forhim')

In [150]:
soup = BeautifulSoup(dr.page_source)

In [151]:
titles = [x.getText(strip=True) for x in soup.findAll('div', {'class': 'w_pro_text17'})]

In [152]:
prod_urls = ["https://www.citychain.com.hk/"+x.find('a')['href']
             for x in soup.findAll('div', {'class': 'w_pro_text17'})]

In [162]:
prod_ids = [x.find('dd').getText() for x in soup.findAll('figure', {'class':'col-xs-6 epro_5'})]

In [153]:
original_prices = []
sale_prices = [] 

In [154]:
for x in soup.findAll('div', {'class': 'w_pro_price17'}):
#     print(x)
    try:
        sale_prices.append(x.find('dd').getText(strip=True))
    except:
        pass
#         try:
#             original_prices.append(x.find('dd').text.strip())
#         except:
#             original_prices.append('N\A')
    try:
        original_prices.append(x.find('del').getText(strip=True))
    except:
        original_prices.append('N\A')

# original_price = [x.find('del').text for x in product_code]
# sale_price = [x.find('dt').text for x in product_code]

In [163]:
len(prod_urls), len(titles), len(original_prices), len(sale_prices)

(20, 20, 20, 20)

In [165]:
dff = pd.DataFrame({
    'ProductName': titles,
    'ProductID': prod_ids,
    'OriginalPrice': original_prices,
    'SalePrice': sale_prices,
    'ProductURL': prod_urls,
})

In [166]:
dff.to_csv('unga.csv', index=False) 

In [473]:
site = requests.get('https://www.citychain.com.hk/en/Product/EshopDetail/class-0/product-907020/cm-0/d-0')

In [474]:
soup = BeautifulSoup(site.content)

In [505]:
all_images_tags = soup.find('ul', {'id':"ulImgList"}).findAll('li')
img_url_1 = all_images_tags[0].find('img')['src'].replace('48x57', '700x650')
try:
    img_url_2 = all_images_tags[1].find('img')['src'].replace('48x57', '700x650')
except:
    img_url_2 = 'N\A'
try:
    img_url_3 = all_images_tags[2].find('img')['src'].replace('48x57', '700x650')
except:
    img_url_3 = 'N\A'
try:
    img_url_4 = all_images_tags[3].find('img')['src'].replace('48x57', '700x650')
except:
    img_url_4 = 'N\A'
try:
    img_url_5 = all_images_tags[4].find('img')['src'].replace('48x57', '700x650')
except:
    img_url_5 = 'N\A'

In [508]:
img_url_1, img_url_2

('https://citychainf02.easesales.com/upload/3F944AB2E078F2BA/202012/4941931521322538056.png_700x650.ashx',
 'https://citychainf01.easesales.com/upload/3F944AB2E078F2BA/202012/5053916301016942403.jpg_700x650.ashx')

In [354]:
soup.find('', {"id":"delPrice"}).text

'HKD 3,290'

In [356]:
title = soup.find('div', {"class": "w_pc_proname"}).find('h3').text.strip()

In [357]:
title

'Seiko Prospex'

In [358]:
prod_id = soup.find('div', {"class": "w_pc_proname"}).find('productno').text

In [359]:
soup.find('', {"id":"delPrice"}).text

'HKD 3,290'

In [360]:
soup.find('', {"id":"dtSalesPrice"}).text.strip()

'HKD 2,632'

In [361]:
table1 = soup.findAll('table', {"class": "product-details-table"})[1]
table2 = soup.findAll('table', {"class": "product-details-table"})[2]

In [362]:
table1_text = table1.getText(separator=" ")

In [363]:
table2_text = table2.getText(separator=" ")

In [364]:
table1_text

'CASE Diameter 46.7 mm Thickness 12.7 mm Shape Round Glass Sapphire Glass DIAL   Color Military Green STRAP   Color Military Green Width 22 mm Material Silicon'

In [365]:
ind1 = table1_text.find('CASE') + 5
ind2 = table1_text.find('DIAL')
case = table1_text[ind1:ind2].replace('\t', '').strip()
case

'Diameter 46.7 mm Thickness 12.7 mm Shape Round Glass Sapphire Glass'

In [366]:
ind1 = table1_text.find('DIAL') + 4
ind2 = table1_text.find('STRAP')
dial = table1_text[ind1:ind2].replace('\t', '').strip()
dial

'Color Military Green'

In [367]:
ind1 = table1_text.find('STRAP') + 5
strap = table1_text[ind1:].replace('\t', '').strip()
strap

'Color Military Green Width 22 mm Material Silicon'

In [377]:
ind1 = table2_text.find('MOVEMENT')+8
ind2 = table2_text.find('WATER RESISTANT')
movement = table2_text[ind1:ind2].replace('\t', '').strip()
movement

'Solar'

In [378]:
table2_text

'MOVEMENT   Solar WATER RESISTANT   20 ATM'

In [379]:
ind1 = table2_text.find('WATER RESISTANT')+18
ind2 = table2_text.find('ESTIMATED')
water_resist = table2_text[ind1:].replace('\t', '').strip()
if ind2 == -1:
    water_resist = table2_text[ind1:].replace('\t', '').strip()
water_resist

'20 ATM'

In [267]:
case

'Diameter       42.5 mm       Thickness       13.4 mm       Shape       Round       Glass       Hardlex Crystal'

In [185]:
titles, prod_ids, orig_prices, sale_prices = [], [], [], []
cases, straps, dials, movements, water_resists = [], [], [], [], []

In [186]:
def scrapFunc(url):
    site = requests.get(url)
    soup = BeautifulSoup(site.content)
    
    title = soup.find('div', {"class": "w_pc_proname"}).find('h3').text.strip()
    prod_id = soup.find('div', {"class": "w_pc_proname"}).find('productno').text
    orig_price = soup.find('del', {"id":"delPrice"}).text
    sale_price = soup.find('', {"id":"dtSalesPrice"}).text.strip()
    
    table1 = soup.findAll('table', {"class": "product-details-table"})[1]
    table1_text = table1.getText(separator=" ")
    
    ind1 = table1_text.find('CASE') + 5
    ind2 = table1_text.find('DIAL')
    case = table1_text[ind1:ind2]
    
    ind1 = table1_text.find('DIAL') + 4
    ind2 = table1_text.find('STRAP')
    dial = table1_text[ind1:ind2].strip()
    
    ind1 = table1_text.find('STRAP') + 5
    strap = table1_text[ind1:].strip()
    
    
    table2 = soup.findAll('table', {"class": "product-details-table"})[2]
    table2_text = table2.getText(separator=" ")
    
    ind2 = table2_text.find('WATER RESISTANT')
    movement = table2_text[8:ind2].strip()
    
    ind1 = table2_text.find('WATER RESISTANT')+18
    water_resist = table2_text[ind1:].strip()
    
    
    titles.append(title); prod_ids.append(prod_id)
    orig_prices.append(orig_price); sale_prices.append(sale_price)
    cases.append(case); dials.append(dial) ; straps.append(strap)
    movements.append(movement); water_resists.append(water_resist)

In [179]:
scrapFunc("https://www.citychain.com.hk/en/Product/EshopDetail/class-0/product-906680/cm-0/d-0")

In [184]:
movements

['Solar']